# ADK Context Deep Dive

A practical, **course-ready** deep dive on how context works in the Agent Development Kit (ADK). 
Use this notebook as your teaching script and as reference material for learners.

**You’ll learn:**
- What *context* means in ADK and why it matters
- The four context objects (InvocationContext, ReadonlyContext, CallbackContext, ToolContext)
- How context flows through an invocation
- How to read/write session state, work with artifacts, request auth, and use memory via context
- Best practices, tables, and quick-reference checklists


## Table of Contents
1. [What is Context?](#what-is-context)
2. [Why Context Matters](#why-context-matters)
3. [Context Flow (High-Level Diagram)](#context-flow)
4. [Types of Context](#types-of-context)
   - [InvocationContext](#invocationcontext)
   - [ReadonlyContext](#readonlycontext)
   - [CallbackContext](#callbackcontext)
   - [ToolContext](#toolcontext)
5. [Common Tasks Using Context](#common-tasks)
   - [Accessing Information](#accessing-information)
   - [Managing State](#managing-state)
   - [Working with Artifacts](#working-with-artifacts)
   - [Handling Tool Authentication](#handling-tool-authentication)
   - [Leveraging Memory](#leveraging-memory)
6. [Advanced: Direct InvocationContext Usage](#advanced-invocation)
7. [Best Practices & Quick Reference](#best-practices)


## What is Context?
<a id='what-is-context'></a>

In ADK, **context** is the bundle of information your agent and tools receive during an interaction (an *invocation*).
It includes the **session**, **state**, **artifacts**, **services**, and identifiers (like **invocation_id**) needed to make informed decisions.

Think of context as the execution envelope for a single user-request-to-final-response cycle.


## Why Context Matters
<a id='why-context-matters'></a>

- **Maintains continuity**: remembers preferences and progress via *session.state*.
- **Shares data across steps**: tools/agents read & write state to cooperate.
- **Grants capabilities**: artifact storage, memory search, auth flows, etc.
- **Aids observability**: context carries IDs for logging & debugging.


## Context Flow (High-Level Diagram)
<a id='context-flow'></a>

```mermaid
flowchart LR
  U[User Input] --> R[(Runner)]
  R -->|Create| C[InvocationContext]
  C --> A[Agent]
  A -->|before_agent/LLM| CB[CallbackContext]
  A -->|LLM calls Tool| T[Tool]
  T -->|ToolContext| TC[State/Artifacts/Auth/Memory]
  A -->|after_agent/LLM| CB2[CallbackContext]
  CB2 --> S[(SessionService)]
  TC --> S
  S --> H[Session History & State]
  A --> R
  R --> U2[Final Response]
```

**Idea:** The Runner creates an **InvocationContext** for each invocation. 
Callbacks see a **CallbackContext**, tools see a **ToolContext** (which extends CallbackContext), and instruction providers use **ReadonlyContext**.


## Types of Context
<a id='types-of-context'></a>

| Context | Where You See It | Can Read State | Can Write State | Artifacts | Auth | Memory | Notes |
|---|---|---:|---:|---:|---:|---:|---|
| **InvocationContext** | Inside agent core (`_run_async_impl`) | ✅ | ⚠️*rare* | via `ctx` services | — | — | Full control for advanced flows |
| **ReadonlyContext** | Instruction providers | ✅ | ❌ | ❌ | ❌ | ❌ | Safe read-only access (no mutations) |
| **CallbackContext** | Agent/model callbacks | ✅ | ✅ | `save_artifact`, `load_artifact` | ❌ | ❌ | Preferred place to mutate state in callbacks |
| **ToolContext** | Tool functions/tool callbacks | ✅ | ✅ | `save/load/list` | `request_credential`/`get_auth_response` | `search_memory` | Adds auth, memory, listing, and `function_call_id` |


### InvocationContext
<a id='invocationcontext'></a>

**Use when:** writing a custom agent or controlling the invocation lifecycle.

**You typically get:**
- `session` (state + history)
- `invocation_id`, `agent` (current agent)
- services like `session_service`, `artifact_service`, optionally `memory_service`
- flags like `end_invocation`

```python
# Pseudocode: custom agent using InvocationContext
from google.adk.agents import BaseAgent
from google.adk.agents.invocation_context import InvocationContext
from google.adk.events import Event
from typing import AsyncGenerator

class ControllerAgent(BaseAgent):
    async def _run_async_impl(self, ctx: InvocationContext) -> AsyncGenerator[Event, None]:
        if ctx.session.state.get("critical_error"):
            ctx.end_invocation = True
            yield Event(author=self.name, content=None)  # signal stop
            return
        # normal work ...
        yield Event(author=self.name, content=None)
```


### ReadonlyContext
<a id='readonlycontext'></a>

**Use when:** providing dynamic instructions safely without mutating state.

```python
from google.adk.agents import LlmAgent
from google.adk.agents.readonly_context import ReadonlyContext

def instruction_provider(ctx: ReadonlyContext) -> str:
    tier = ctx.state().get("user:tier", "standard")
    return f"Answer succinctly for a {tier} user."

agent = LlmAgent(
    name="dynamic_instructions",
    model="gemini-2.0-flash",
    instruction=instruction_provider,
)
```


### CallbackContext
<a id='callbackcontext'></a>

**Use when:** reading/writing state or artifacts inside callbacks (before/after agent or model).

```python
from google.adk.agents.callback_context import CallbackContext
from google.adk.models import LlmRequest
from typing import Optional

def before_model_cb(ctx: CallbackContext, req: LlmRequest) -> Optional[None]:
    calls = ctx.state.get("model_calls", 0)
    ctx.state["model_calls"] = calls + 1  # tracked and persisted
    return None  # proceed with model call
```


### ToolContext
<a id='toolcontext'></a>

**Use when:** implementing a tool. Adds auth, memory search, and artifact listing on top of CallbackContext.

```python
from google.adk.tools.tool_context import ToolContext

def fetch_orders(user_id: str, tool_context: ToolContext) -> dict:
    # Read from state
    preferred_region = tool_context.state.get("user:region", "us-east")
    # Optionally search memory
    # mem = tool_context.search_memory(f"recent orders for {user_id}")
    return {"region": preferred_region, "orders": ["o-1", "o-2"]}
```


## Common Tasks Using Context
<a id='common-tasks'></a>
Below are core operations you’ll perform with context.


### Accessing Information
<a id='accessing-information'></a>

```python
# In a Tool
from google.adk.tools.tool_context import ToolContext

def show_ids(tool_context: ToolContext) -> dict:
    return {
        "agent": tool_context.agent_name,
        "invocation_id": tool_context.invocation_id,
        "has_function_call_id": bool(getattr(tool_context, "function_call_id", None)),
    }
```


### Managing State
<a id='managing-state'></a>

Use **CallbackContext.state**/**ToolContext.state** for tracked updates. These changes are recorded as `EventActions.state_delta`.

```python
from google.adk.tools.tool_context import ToolContext

def set_user_pref(tool_context: ToolContext, key: str, value: str) -> dict:
    tool_context.state[f"user:{key}"] = value  # persists with persistent session service
    return {"ok": True}
```


### Working with Artifacts
<a id='working-with-artifacts'></a>

Artifacts are saved/loaded via context—perfect for files or binary data.

```python
import google.genai.types as types
from google.adk.agents.callback_context import CallbackContext

async def save_report(ctx: CallbackContext, data: bytes) -> int:
    part = types.Part.from_bytes(data=data, mime_type="application/pdf")
    version = await ctx.save_artifact("report.pdf", part)
    return version
```


### Handling Tool Authentication
<a id='handling-tool-authentication'></a>

Use **ToolContext.request_credential(...)** to start auth flows and **get_auth_response(...)** to retrieve credentials.

```python
from google.adk.tools.tool_context import ToolContext
# from google.adk.auth import AuthConfig  # example placeholder

def call_secure_api(tool_context: ToolContext) -> dict:
    token = tool_context.state.get("user:my_api_token")
    if not token:
        # auth_config = AuthConfig(...)
        # tool_context.request_credential(auth_config)
        return {"status": "auth_required"}
    return {"status": "ok", "preview": "fetched data"}
```


### Leveraging Memory
<a id='leveraging-memory'></a>

If a `memory_service` is configured, tools can search it via **ToolContext.search_memory(...)**.

```python
from google.adk.tools.tool_context import ToolContext

def find_related(tool_context: ToolContext, topic: str) -> dict:
    # results = tool_context.search_memory(f"about {topic}")
    # return {"found": len(results.results) if results else 0}
    return {"info": "memory search example (configure service to enable)"}
```


## Advanced: Direct InvocationContext Usage
<a id='advanced-invocation'></a>

Occasionally you’ll use **InvocationContext** directly in a custom agent to control the run (e.g., early termination, branching). Prefer callbacks/tools for most state and artifact work.

```python
from google.adk.agents import BaseAgent
from google.adk.agents.invocation_context import InvocationContext
from google.adk.events import Event
from typing import AsyncGenerator

class StopOnFlagAgent(BaseAgent):
    async def _run_async_impl(self, ctx: InvocationContext) -> AsyncGenerator[Event, None]:
        if ctx.session.state.get("temp:stop"):
            ctx.end_invocation = True
            yield Event(author=self.name)
            return
        yield Event(author=self.name)
```


## Best Practices & Quick Reference
<a id='best-practices'></a>

### Design Principles
- **Use the narrowest context**: ToolContext for tools, CallbackContext for callbacks.
- **State flow via context**: prefer `ctx.state[...] = ...` in callbacks/tools rather than editing sessions directly.
- **Artifacts for binary data**: `save_artifact` / `load_artifact` in context, not in state.
- **Auth via ToolContext**: request & retrieve credentials in the tool that needs them.
- **Memory is optional**: check availability before calling.

### Quick Cheatsheet

| Need | Use |
|---|---|
| Read user/app flags | `ctx.state.get(...)` |
| Persist a file | `ctx.save_artifact(filename, part)` |
| Load last file version | `ctx.load_artifact(filename)` |
| Ask for credentials | `tool_context.request_credential(...)` |
| Search memory | `tool_context.search_memory(query)` |
| End the whole turn early | `invocation_ctx.end_invocation = True` |

```mermaid
flowchart TD
  A[CallbackContext.state write] --> D[EventActions.state_delta]
  T[ToolContext.save_artifact] --> AD[EventActions.artifact_delta]
  D --> S[(SessionService applies deltas)]
  AD --> S
  S --> H[Session.state & Artifacts Updated]
```
